In [442]:
import os

import numpy as np

from matplotlib import pyplot

from scipy import optimize

from sklearn.model_selection import train_test_split

%matplotlib inline

In [443]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [444]:
!ls
%mkdir data
!ls

data  gdrive  sample_data
mkdir: cannot create directory ‘data’: File exists
data  gdrive  sample_data


In [445]:
import sys

sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks/datasets')
sys.path.insert(1, '/content/gdrive/MyDrive/Colab Notebooks/Laboratorios SIS 420/Lab5')

In [446]:
import shutil

shutil.copy('/content/gdrive/MyDrive/Colab Notebooks/Datasets/l4d2_player_stats_preparado.csv', '/content/data')

'/content/data/l4d2_player_stats_preparado.csv'

In [447]:
data = np.loadtxt("/content/data/l4d2_player_stats_preparado.csv", delimiter=',',skiprows=1)

X, y = data[:, 0:], data[:,111]

y = np.array([int(e) for e in y])
print(y.shape)
y = np.squeeze(y)

y[y == 1] = 0
y[y == 2] = 1
y[y == 3] = 2
y[y == 4] = 3

X, Xp, y, yp = train_test_split(X, y, test_size=0.2, random_state=42)

print(X.shape)
print(y.shape)
print(Xp.shape)
print(yp.shape)

print(y)
print(yp)

(20830,)
(16664, 112)
(16664,)
(4166, 112)
(4166,)
[2 3 3 ... 0 1 1]
[3 1 2 ... 2 3 3]


In [448]:
input_layer_size  = 112
hidden_layer_size = 35
num_labels = 4

pesos = {}
pesos['Theta1'] = np.random.rand(35, 113)
pesos['Theta2'] = np.random.rand(4, 36)

Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']

print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(3955,)
(144,)
(4099,)


In [449]:
def sigmoid(z):

    return 1.0 / (1.0 + np.exp(-z))


def sigmoidGradient(z):

    g = np.zeros(z.shape)

    g = sigmoid(z) * (1 - sigmoid(z))

    return g

In [450]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_= 0.0):

    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size

    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)

    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)

    a3 = sigmoid(a2.dot(Theta2.T))

    y_matrix = y.reshape(-1)

    y_matrix = np.eye(num_labels)[y_matrix]

    temp1 = Theta1
    temp2 = Theta2

    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))

    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term


    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]

    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]


    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

In [451]:
lambda_ = 0
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_)
print('Costo en parametros (cargado de ex4weights): %.6f ' % J)
print('El costo debe esta cercano a               : 0.287629')

Costo en parametros (cargado de ex4weights): 53.085840 
El costo debe esta cercano a               : 0.287629


In [452]:
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:\n  ')
print(g)

Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:
  
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [453]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):

    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [454]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [455]:
options= {'maxiter': 1000}

lambda_ = 1


costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

nn_params = res.x

Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

<ipython-input-455-7edff579fb0d>:10: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(costFunction,
<ipython-input-449-6b5d5f8d5f68>:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [456]:
def predict(Theta1, Theta2, X):

    m = X.shape[0]
    num_labels = Theta2.shape[0]

    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [457]:
pred = predict(Theta1, Theta2, X[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == y[:]) * 100))

[1 3 3 ... 0 1 1]
Training Set Accuracy: 50.876140


<ipython-input-449-6b5d5f8d5f68>:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [458]:
pred = predict(Theta1, Theta2, Xp[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == yp[:]) * 100))

[1 1 1 ... 1 1 3]
Training Set Accuracy: 49.471916


<ipython-input-449-6b5d5f8d5f68>:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


Se separaron los datos en 80% para entrenamiento y 20% para pruebas, usando pandas

Se probaron varios hiperparametros para obtener los mejores resultados
se uso varios hidden layers, pero el tiempo de carga se elevo demasiado enmuchas ocaciones, llegando a la conclusion de:
hidden_layer: debe ser superior a 15 y menor que 30, ya que no se observan mejoras a fuera de ahi.
lamba: no mejora mucho los resultados y los empeora a partir del valor 3.